In [12]:
import urllib.request
import json
import os
import ssl

In [15]:
from dotenv import dotenv_values
from dotenv import load_dotenv

In [16]:
config = dotenv_values(".env")

In [17]:
# Get the model object from HuggingFaceHub. We can use it to check for sample test data
import urllib.request, json

raw_data = urllib.request.urlopen(
    "https://huggingface.co/api/models/" + 'deepset/roberta-base-squad2'
)

print("https://huggingface.co/api/models/" + 'deepset/roberta-base-squad2')
data = json.load(raw_data)
print(data)

https://huggingface.co/api/models/deepset/roberta-base-squad2
{'_id': '621ffdc136468d709f17a5fd', 'id': 'deepset/roberta-base-squad2', 'modelId': 'deepset/roberta-base-squad2', 'author': 'deepset', 'sha': 'e84d19c1ab20d7a5c15407f6954cef5c25d7a261', 'lastModified': '2023-09-26T11:36:30.000Z', 'private': False, 'disabled': False, 'gated': False, 'pipeline_tag': 'question-answering', 'tags': ['transformers', 'pytorch', 'tf', 'jax', 'rust', 'safetensors', 'roberta', 'question-answering', 'en', 'dataset:squad_v2', 'license:cc-by-4.0', 'model-index', 'endpoints_compatible', 'has_space', 'region:us'], 'downloads': 1217829, 'library_name': 'transformers', 'mask_token': '<mask>', 'widgetData': [{'text': 'Where do I live?', 'context': 'My name is Wolfgang and I live in Berlin'}, {'text': 'Where do I live?', 'context': 'My name is Sarah and I live in London'}, {'text': "What's my name?", 'context': 'My name is Clara and I live in Berkeley.'}, {'text': 'Which name is also used to describe the Amaz

In [18]:
data.keys()

dict_keys(['_id', 'id', 'modelId', 'author', 'sha', 'lastModified', 'private', 'disabled', 'gated', 'pipeline_tag', 'tags', 'downloads', 'library_name', 'mask_token', 'widgetData', 'likes', 'model-index', 'config', 'cardData', 'transformersInfo', 'spaces', 'safetensors', 'siblings', 'createdAt'])

In [19]:
data['widgetData']

[{'text': 'Where do I live?',
  'context': 'My name is Wolfgang and I live in Berlin'},
 {'text': 'Where do I live?',
  'context': 'My name is Sarah and I live in London'},
 {'text': "What's my name?",
  'context': 'My name is Clara and I live in Berkeley.'},
 {'text': 'Which name is also used to describe the Amazon rainforest in English?',
  'context': 'The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colo

In [20]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

In [22]:
api_key = config.get("KEY")

In [23]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
  "inputs": {
    "question": "Which name is also used to describe the Amazon rainforest in English?",
    "context": "The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain 'Amazonas' in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.",
  }
}

body = str.encode(json.dumps(data))

In [24]:
url = 'https://question-answering-1709667031.uksouth.inference.ml.azure.com/score'
# https://question-answering-1709656967.uksouth.inference.ml.azure.com/score
# Replace this with the primary/secondary key or AMLToken for the endpoint
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'test-qa' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'{"score":0.7497807741165161,"start":201,"end":230,"answer":"Amazonia or the Amazon Jungle"}'


In [25]:
print(result.decode("utf8", 'ignore'))

{"score":0.7497807741165161,"start":201,"end":230,"answer":"Amazonia or the Amazon Jungle"}
